# Convolutional Neural Networks
Convolutional Neural Networks( or CovNets) are mainly used for image classification. They are also used for NLP. But in this notebook we'll classify handwritten digts (MNIST). Let's dive right in! 
#### MNIST Dataset Example
![MNIST_Dataset](https://upload.wikimedia.org/wikipedia/commons/thumb/2/27/MnistExamples.png/220px-MnistExamples.png)

In this tutorial, we're gonna build this model with tensorflow. Tensorflow is a library to build models. We don't have that much time to look into math and build models. It makes our work easier. Calculating hidden errors. etc.. The tensoflow API is really beautiful. You'll love it! Let's get into the code 
![](http://klaatuveratanecto.com/wp-content/uploads/2017/09/markblogtensorflow.png)

Best Analogy for using tensorflow
-->
###### If you wanna build something out of lego you don't build the lego blocks, instead what you do is you buy the lego set and build something amazing
##### - Future Daniel Bourke
Chekout his [channel](https://www.youtube.com/channel/UCr8O8l5cCX85Oem1d18EezQ) for AI vlogs, Resources, ways to learn and so much more! 

In [4]:
pip install tensorflow # To install tensorflow.This should be done in cmd or a terminal


The following command must be run outside of the IPython shell:

    $ pip install tensorflow # To install tensorflow.This should be done in cmd or a terminal

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/


In [2]:
import tensorflow as tf #Importing the and calling it tf so that we dont need to write a big name everytime~tensorflow
#(We're lazy)
from tensorflow.examples.tutorials.mnist import input_data #tensorflow readily provides the dataset
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True) #storing the dataset in a variable

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


We also set one_hot to True
What's one_hot? Checkout this explanation
--> https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f

#### New to tensorflow? 
If you're new to tensorflow checkout this 
--> https://www.datacamp.com/community/tutorials/tensorflow-tutorial

Given these images, our neural net will be able classify and give us the probability of each image being a certain class. Our network should look like this 
![](https://www.mathworks.com/content/mathworks/www/en/discovery/convolutional-neural-network/jcr:content/mainParsys/image_copy.adapt.full.high.jpg/1508999490138.jpg)

In [3]:
n_classes = 10 #Number of classes
# Let's define our weights
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
x = tf.placeholder(tf.float32, [None, 784]) #None indicates batch_size which is the number of samples you train your model
# ^ is for the data(Images)
y = tf.placeholder(tf.float32, [None, n_classes])#True output
#Creating placeholders will let us define them while creating a tensorflow session

### WHY CONVNETS?
Using regular neural network( Ive written about them in [this](https://github.com/Developer404/Neural_Network) notebook) is fine but you require more computational power( for 1080p images, you require 1166400 weights!!). We dont have that much computational power

# Architecture
#### ~Convolution
Instead of fully connected layers. We us ecovnets, Input is a image.
For a computer the image is just an array of numbers with no meaning in it. It's just a matrix of numbers
![](http://blog.kleinproject.org/wp-content/uploads/2012/04/sample-matrix.gif)

we take a kernel(aka filters)[Weights] of some size and multiply(and add) it all over the image( Element wise multiplication) Like this. 
##### Gif that shows convolution with a stride of 1 ![](http://deeplearning.net/software/theano/_images/numerical_no_padding_no_strides.gif)

#### Some terms to Remember ;)
###### Stride: 
It's the number of times we move the filter[aka Weights] on the image.
###### Zero Padding:
Sometimes we don't use convolution to make the image smaller. So, we use zero padding to avoid that by padding the borders by zero ie.. Putting zeros around the image and then performing convolution. Here's a gif
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2017/06/28094927/padding.gif).
And we get the output size same as the input size (without zero-padding).
###### Valid padding:
Valid padding can be referred to as noremal convolution. You don't but zeros around the borders of the image

#### ~Relu
As we perform activation in regural nets, we do it here too. We perform relu activation and what it does is very simple , it just turns the negative values to 0. And keeps the rest of the postive numbers same
![](https://lazyprogrammer.me/wp-content/uploads/2017/10/relu.png)

In [15]:
def conv2d(x, W, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME"), b))

#### ~Maxpooling
Maxpooling is a way to reduce dimension=[computational complexity] as it selects the largest element from the window of size that we define. And we also define the number of strides
![](https://shafeentejani.github.io/assets/images/pooling.gif)

In [21]:
def maxpool2d(x, ksize=2):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')

### Let's get into the code!

In [ ]:
#Hyperparameters
batch_size = 128 #The number of samples we want our model to train on 
learning_rate = 0.5#How fast our model coverges
training_iters = 300000#training iterations
display_step = 10#How often we want to display the accuracy
def model(x, weights, biases):
    x = tf.reshape(x, [-1, 28, 28, 1])
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    maxpool1 = maxpool2d(conv1)
    conv2 = conv2d(maxpool1, weights['wc2'], biases['bc2'])
    maxpool2 = maxpool2d(conv2)
    maxpool2 = tf.layers.flatten(maxpool2) 
    fc1 = tf.add(tf.matmul(maxpool2, weights['wd1']),biases['bd1'])
    logits = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return logits
pred = model(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) 
# Initializing the variables
init = tf.initialize_all_variables()
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       })
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              })
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 1280, Minibatch Loss= 651.818054, Training Accuracy= 0.14062
Iter 2560, Minibatch Loss= 747.345459, Training Accuracy= 0.08594
Iter 3840, Minibatch Loss= 202.348450, Training Accuracy= 0.06250
Iter 5120, Minibatch Loss= 154.696594, Training Accuracy= 0.10938
Iter 6400, Minibatch Loss= 138.584885, Training Accuracy= 0.07812
Iter 7680, Minibatch Loss= 54.935440, Training Accuracy= 0.10938
Iter 8960, Minibatch Loss= 133.113525, Training Accuracy= 0.08594
Iter 10240, Minibatch Loss= 243.589310, Training Accuracy= 0.10156
Iter 11520, Minibatch Loss= 258.163239, Training Accuracy= 0.10156
Iter 12800, Minibatch Loss= 515.287720, Training Accuracy= 0.07031
Iter 14080, Minibatch Loss= 210.003235, Training Accuracy= 0.08594
Iter 15360, Minibatch Loss= 330.890869, Training Accuracy= 0.10938
Iter 16640, Minibatch Loss= 181.708038, Training Accuracy= 0.11719
Iter 17920, Minibatch Loss= 136.839355, Training Accuracy= 0.08594
Iter 19200, Minibatch Loss= 107.965088, Training Accuracy= 0.08594
Ite

### Beauty of backpropagation
While training, our model learns features of images. When we printed out the feature maps(Output of convolution). We found out that These are the same images but a little blurred. Checkout [this](http://cs231n.stanford.edu/) neural network on the top of the page which gives the feature maps. We refers to researchers mainly.. Yann Lecun. Who is one of inspirations!
![](http://www.i-programmer.info/images/stories/News/2016/Nov/A/yannpic.jpg)

This should be understandable but still confusing. I'm here to help. One of my new year's resolution was to make youtube videos on my [channel](https://www.youtube.com/channel/UCFViGhziBdf6BDYIkeQeqQw). And I promise I'm gonna try my best to explain. And the explanations will be much more clearer. I hope you liked the Notebook! You could ask me questions on this notebook and also can suggest what changes I have to make and recommend new topics and just become friends by emailing me at shaikasad17@gmail.com
And I'll try my best to reply! 
Youtube videos coming soon..
If you want to get started I'd recommend my last [repo](github.com/Developer404/Neural_Network) for understanding simple feed forward neural networks
And until then, Cheerio!

BTW, I did not train the network fully and go ahead and train it fully by running the cell!. The accuracy is also pretty bad I'm gonna update the notebook by Discussing some techniques of how to prevent overfitting and improve our model by applying dropout, regularization, etc..